In [1]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np

# from sklearn import linear_model
# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
# from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import KFold
from itertools import combinations
from itertools import permutations
import matplotlib.pyplot as plt
%matplotlib inline

def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'NaN', 1 : '% of Total Values'})
        return mis_val_table_ren_columns

def profile(df):
    for column in df:
        print(column, '------------------------------------------------')
        print(df[column].value_counts(normalize=True, sort=True, ascending=False, bins=None, dropna=False))

In [2]:
#df = pd.read_csv('mnist.csv', index_col=0)
#df = pd.read_csv('lemon_training.csv', names = ['Age','WorkClass','FnlWgt','Edu','EduNum','Married','Occ','Relationship','Race','Sex','CapGain','CapLoss','HrsWk','Country','Income'])
df = pd.read_csv('lemon_training.csv')
df.head()

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020
3,6,0,12/7/2009,ADESA,2004,5,MITSUBISHI,GALANT 4C,ES,4D SEDAN ES,...,8149.0,9451.0,NaN,NaN,19638,33619,FL,5600.0,0,594
4,7,0,12/7/2009,ADESA,2004,5,KIA,SPECTRA,EX,4D SEDAN EX,...,6230.0,8603.0,NaN,NaN,19638,33619,FL,4200.0,0,533


In [ ]:
df.info()
df.VehOdo.hist()
profile(X)
missing_values_table(df)

In [ ]:
df.MMRAcquisitionAuctionAveragePrice.hist()

In [ ]:
#df.VehBCost.sort_values()
#df.MMRAcquisitionAuctionAveragePrice.describe()

In [45]:
#df[['VehYear','VehicleAge','MMRCurrentRetailAveragePrice','MMRCurrentRetailCleanPrice','VNZIP1','VNST','VehBCost','IsOnlineSale','WarrantyCost'
#,'AuctionCat','MakeCat','ColorCat','TransCat','WheelTypeCat','VehOdo','NatCat','Big3Cat','PrimeCat','AucGuartCat']]
#df['VehAge'] = pd.to_datetime(df.PurchDate).dt.year - df.VehYear
df['MakeCat'] = df.Make.astype('category', ordered=True).cat.codes
df['AuctionCat'] = df.Auction.astype('category',ordered=True).cat.codes
df['ColorCat'] = df.Color.astype('category',ordered=True).cat.codes
df['TransCat'] = df.Transmission.astype('category',ordered=True).cat.codes
df['WheelType'] = df['WheelType'].fillna('Unknown')
df['WheelTypeCat'] = df.WheelType.astype('category',ordered=True).cat.codes
df['NatCat'] = df.Nationality.astype('category',ordered=True).cat.codes
df['Big3Cat'] = df.TopThreeAmericanName.astype('category',ordered=True).cat.codes
df['PrimeCat'] = df.PRIMEUNIT.astype('category',ordered=True).cat.codes
df['AucGuartCat'] = df.AUCGUART.astype('category',ordered=True).cat.codes
df['ZipCat'] = df.VNZIP1.astype('category',ordered=True).cat.codes
df['StateCat'] = df.VNST.astype('category',ordered=True).cat.codes

df['MMRAcquisitionAuctionAveragePrice'] = df['MMRAcquisitionAuctionAveragePrice'].fillna(df.VehBCost)
df['MMRAcquisitionAuctionCleanPrice'] = df['MMRAcquisitionAuctionCleanPrice'].fillna(df.VehBCost)
df['MMRAcquisitionRetailAveragePrice'] = df['MMRAcquisitionRetailAveragePrice'].fillna(df.VehBCost)
df['MMRAcquisitonRetailCleanPrice'] = df['MMRAcquisitonRetailCleanPrice'].fillna(df.VehBCost)
df['MMRCurrentAuctionAveragePrice'] = df['MMRCurrentAuctionAveragePrice'].fillna(df.VehBCost)
df['MMRCurrentAuctionCleanPrice'] = df['MMRCurrentAuctionCleanPrice'].fillna(df.VehBCost)
df['MMRCurrentRetailAveragePrice'] = df['MMRCurrentRetailAveragePrice'].fillna(df.VehBCost)
df['MMRCurrentRetailCleanPrice'] = df['MMRCurrentRetailCleanPrice'].fillna(df.VehBCost)

df['MnthPurch'] = pd.to_datetime(df.PurchDate).dt.month
#df['OverPaid'] = df.VehBCost.apply(lambda x : 1 if x > (df['MMRAcquisitionAuctionAveragePrice'] + 500) else 0)
#df['UnderPaid'] = df.VehBCost.apply(lambda x : 1 if x < df.MMRAcquisitionAuctionAveragePrice - 500. else 0)
df['OverPaid'] = np.where(df['VehBCost'] > df['MMRAcquisitionAuctionAveragePrice'] + 500, 1, 0)
df['UnderPaid'] = np.where(df['VehBCost'] < df['MMRAcquisitionAuctionAveragePrice'] - 500, 1, 0)
#df['CostFairDiff'] = 
#df['PRetail']

# df['HasCabin'] = df["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
# df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
# df['IsAlone'] = df.FamilySize.apply(lambda x: 1 if x == 1 else 0)
# df['Embarked'] = df['Embarked'].fillna('S')
# df['Embarked'] = df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} )
# df['Age'] = df['Age'].fillna(df.Age.mean())
# df['Age'] = df.Age.apply(lambda x: 0 if x < 16 else 1 if x < 32 else 2 if x < 48 else 3 if x < 64 else 4)
# df['Fare'] = df.Fare.apply(lambda x : 0 if x < 7.91 else 1 if x < 14.45 else 2 if x < 31 else 3)
# df['IsMale'] = df['Sex'].map({' Male':1, ' Female':0})
# df['IsWhite'] = df.Race.apply(lambda x: 1 if x == ' White' else 0)
# df['IsUSA'] = df.Country.apply(lambda x: 1 if x == ' United-States' else 0)
df.head(5)

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,WheelTypeCat,NatCat,Big3Cat,PrimeCat,AucGuartCat,ZipCat,StateCat,MnthPurch,OverPaid,UnderPaid
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,0,2,3,-1,-1,47,5,12,0,1
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,0,0,0,-1,-1,47,5,12,1,0
2,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,1,0,1,-1,-1,47,5,12,0,0
3,6,0,12/7/2009,ADESA,2004,5,MITSUBISHI,GALANT 4C,ES,4D SEDAN ES,...,1,2,3,-1,-1,47,5,12,1,0
4,7,0,12/7/2009,ADESA,2004,5,KIA,SPECTRA,EX,4D SEDAN EX,...,1,2,3,-1,-1,47,5,12,1,0


In [ ]:
y = df.IsBadBuy
X = df[['VehYear','VehicleAge','ZipCat','StateCat','VehBCost','IsOnlineSale','WarrantyCost','AuctionCat','MakeCat','ColorCat','TransCat'
        ,'WheelTypeCat','VehOdo','NatCat','Big3Cat','MnthPurch'
        ,'OverPaid','UnderPaid','MMRAcquisitionAuctionAveragePrice'
        ,'MMRAcquisitionAuctionCleanPrice','MMRAcquisitionRetailAveragePrice'
        ,'MMRAcquisitonRetailCleanPrice','MMRCurrentAuctionAveragePrice'
        ,'MMRCurrentAuctionCleanPrice','MMRCurrentRetailAveragePrice'
        ,'MMRCurrentRetailCleanPrice']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
splits = 10
cv = KFold(n_splits=splits)
#ls = np.linspace(4,10,5,dtype=int)
d = np.array([12])
e = np.array([10])
f = np.array([10,15,20])
for feat in f:
    for est in e:
        #for depth in range(1, 20):
        for i,depth in enumerate(d):
            model = GradientBoostingClassifier(n_estimators=est,max_depth=depth,learning_rate=.01,max_features=feat).fit(X_train,y_train)
            train_scores, test_scores = [], []
            for train_i, test_i in cv.split(X):
#                 Xr, yr, Xt, yt = X[train_i], y[train_i], X[test_i], y[test_i]  #for arrays
                Xr, yr, Xt, yt = X.loc[train_i], y.loc[train_i], X.loc[test_i], y.loc[test_i]  #for dataframes
                model.fit(Xr, yr)
                test_scores.append(model.score(Xt, yt))
                train_scores.append(model.score(Xr, yr))
            print('test scores:', sum(test_scores)/len(test_scores), 'train scores:', sum(train_scores)/len(train_scores), '-- splits:',splits, 'feat:', feat, 'est:', est, 'depth:', depth, )

In [ ]:
test scores: 0.89647304272 train scores: 0.898010404706 -- splits: 10 feat: 10 est: 500 depth: 2
test scores: 0.898430399744 train scores: 0.901883896946 -- splits: 10 feat: 10 est: 500 depth: 4
test scores: 0.898293363639 train scores: 0.906155391722 -- splits: 10 feat: 10 est: 500 depth: 6
test scores: 0.898489092827 train scores: 0.912349498851 -- splits: 10 feat: 10 est: 500 depth: 8

In [46]:
y = df.IsBadBuy
X = df[['VehYear','VehicleAge','ZipCat','StateCat','VehBCost','WarrantyCost','AuctionCat','MakeCat','ColorCat','WheelTypeCat','VehOdo','Big3Cat','MnthPurch','MMRAcquisitionAuctionAveragePrice','MMRAcquisitionAuctionCleanPrice','MMRAcquisitionRetailAveragePrice','MMRAcquisitonRetailCleanPrice','MMRCurrentAuctionAveragePrice','MMRCurrentAuctionCleanPrice','MMRCurrentRetailAveragePrice','MMRCurrentRetailCleanPrice']]

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
est=1000
depth='auto'
max_feat= 'default' #len(X.columns)
model = GradientBoostingClassifier(n_estimators=est,learning_rate=.007).fit(X_train,y_train)
#print(model.score(X_train,y_train))
#print(model.score(X_test,y_test))
print('test scores:', model.score(X_test,y_test), 'train scores:', model.score(X_train,y_train), '-- splits:',0, 'max_feat:', max_feat, 'n_est:', est, 'depth:', depth, )

test scores: 0.901897983393 train scores: 0.900782984691 -- splits: 0 max_feat: default n_est: 1000 depth: auto


In [52]:
model.feature_importances_

array([ 0.00522927,  0.06512015,  0.11827991,  0.0212318 ,  0.0854868 ,
        0.04664583,  0.05661516,  0.01657341,  0.01262171,  0.26851441,
        0.05683577,  0.01556613,  0.04254798,  0.00821759,  0.02240454,
        0.03445055,  0.04684117,  0.01901499,  0.0172578 ,  0.02021061,
        0.02033444])

In [53]:
X.columns

Index(['VehYear', 'VehicleAge', 'ZipCat', 'StateCat', 'VehBCost',
       'WarrantyCost', 'AuctionCat', 'MakeCat', 'ColorCat', 'WheelTypeCat',
       'VehOdo', 'Big3Cat', 'MnthPurch', 'MMRAcquisitionAuctionAveragePrice',
       'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice',
       'MMRAcquisitonRetailCleanPrice', 'MMRCurrentAuctionAveragePrice',
       'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailAveragePrice',
       'MMRCurrentRetailCleanPrice'],
      dtype='object')